Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию).<br> Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). <br>Получившийся список должен содержать в себе минимум:<br>
1) Наименование вакансии.<br>
2) Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).<br>
3) Ссылку на саму вакансию.<br>
4) Сайт, откуда собрана вакансия.<br><br>
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). <br>Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.<br>

In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
def comp(string):
    data = {}
    x = string.split()
    if len(x) == 0:
        data['min'] = 'NaN'
        data['max'] = 'NaN'
        data['cur'] = 'NaN'
        return data
    if x[0] == 'от':
        data['min'] = x[1]
        data['max'] = 'NaN'
        data['cur'] = x[-1]
        return data
    if x[0] == 'до':
        data['min'] = 'NaN'
        data['max'] = x[1]
        data['cur'] = x[-1]
        return data
    try:
        int(x[0])
        data['min'] = x[0]
        data['max'] = x[2]
        data['cur'] = x[-1]
        return data
    except ValueError:
        print('ошибка при обработке списка ',x)
        print('??????????')

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}
sity = 'saratov'
base_url_hh = f'https://{sity}.hh.ru/'
# https://www.superjob.ru/vacancy/search/?keywords=python%20junior&page=1
base_url_sj = 'https://www.superjob.ru/'
search_hh = 'search/vacancy?area=&fromSearchLine=true&text='
search_sj = 'vacancy/search/?keywords='
search_input_hh = 'Python+junior'
search_input_sj = 'python'
page = '&page='
#search/vacancy?area=&fromSearchLine=true&text=Python+junior&page=0&hhtmFrom=vacancy_search_list

In [4]:
response = requests.get(base_url_hh+search_hh+search_input_hh, headers=headers)

In [5]:
dom = BeautifulSoup(response.text, 'html.parser')

In [6]:
# основная идея это выдирать отдельные данные. 
# сначала выдрать наименование list1
# отдельно выдрать цифры з/п list2
# отдельно получить ссылки list 3
# тут можно написать вручную, можно выдрать из ссылки, которые получили ранее
# дальше это всё записать в одну структуру
# чтобы не делать три цикла можно собирать данные в один проход?
# vacancy_list = dom.find_all('a', {'data-qa': 'vacancy-serp__vacancy-title'})
# список вакансий
# vacancy_comp = dom.find_all('span', {'data-qa':"vacancy-serp__vacancy-compensation"})
# [el.text.replace('\u202f','') for el in vacancy_comp] список зарплаты
# разделить на три поля
# ссылку и сайт
# ссылки: 
# for el in vacancy_list:
#     print(el['href'])
# всё же нужно рыться в одном блоке - теряются данные о зарплате, если она не указана совсем. 
# + такие пропуски смещают список и вакансия может получить неверные данные


vacancy_data = dom.find_all('div', {'class':'vacancy-serp-item'})
n = int(dom.find_all('a', {'class': 'bloko-button'})[-2].text)
n

17

In [7]:
# vacancy_data[1].find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
# vacancy_data[1].find('span', {'data-qa':"vacancy-serp__vacancy-compensation"}).text.replace('\u202f','')
# vacancy_data[1].find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
vacancy_list = []
for i in range(int(dom.find_all('a', {'class': 'bloko-button'})[-2].text)):
    response = requests.get(f'{base_url_hh}{search_hh}{search_input_hh}{page}{i}', headers=headers)
#     print(f'{base_url}{search_hh}{search_input_hh}{page}{i}')
#     print(f'{i}')
    dom = BeautifulSoup(response.text, 'html.parser')
    vacancy_data = dom.find_all('div', {'class':'vacancy-serp-item'})
    for el in vacancy_data:
        temp_data ={}
        vacancy_title = el.find('a', {'data-qa': 'vacancy-serp__vacancy-title'}).text
        try:
            vacancy_comp = el.find('span', {'data-qa':"vacancy-serp__vacancy-compensation"}).text.replace('\u202f','')
        except AttributeError:
            vacancy_comp = ''
        vacancy_href = el.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
        temp_data['title'] = vacancy_title
        temp_data['min_salary'] = comp(vacancy_comp)['min']
        temp_data['max_salary'] = comp(vacancy_comp)['max']        
        temp_data['currency'] = comp(vacancy_comp)['cur']
        temp_data['href'] = vacancy_href
        if vacancy_href.find('hh'):
            temp_data['site'] = 'HeadHunter'
        else: 
            temp_data['site'] = 'Superjob'
        vacancy_list.append(temp_data)

In [8]:
vacancy_list, #len(vacancy_list)

([{'title': 'Python Developer',
   'min_salary': '2500',
   'max_salary': '4500',
   'currency': 'USD',
   'href': 'https://saratov.hh.ru/vacancy/52558321?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
   'site': 'HeadHunter'},
  {'title': 'Junior Python разработчик',
   'min_salary': '40000',
   'max_salary': '100000',
   'currency': 'руб.',
   'href': 'https://saratov.hh.ru/vacancy/52377613?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
   'site': 'HeadHunter'},
  {'title': 'Junior разработчик Python Django',
   'min_salary': '40000',
   'max_salary': '80000',
   'currency': 'руб.',
   'href': 'https://saratov.hh.ru/vacancy/52203762?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=Python%20junior',
   'site': 'HeadHunter'},
  {'title': 'Junior Data Engineer (Python)',
   'min_salary': '150000',
   'max_salary': '190000',
   'currency': 'руб.',
   'href': 'https://saratov.hh.ru/vacancy/50637406?from=vacancy_searc

In [9]:
# response = requests.get(base_url_sj+search_sj+search_input_sj+page+'1', headers=headers)
# dom = BeautifulSoup(response.text, 'html.parser')
# print(base_url_sj+search_sj+search_input_sj+page+'1')
# vacancy_data = dom.find_all('div', {'class':'vacancy-serp-item'})
# n = int(dom.find_all('a', {'class': 'bloko-button'})[-2].text)
# n
# vacancy_data[1].find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
# vacancy_data[1].find('span', {'data-qa':"vacancy-serp__vacancy-compensation"}).text.replace('\u202f','')
# vacancy_data[1].find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
# f-test-search-result-item


In [10]:
# pprint(dom.find_all('div', {'class': '_3zucV _3bZkM _1hWf- _1PXgQ _3bO-Z l9LnJ'}))

In [15]:
vacancy_df = pd.DataFrame(vacancy_list)
vacancy_df

,title,min_salary,max_salary,currency,href,site
0,Python Developer,2500,4500,USD,https://saratov.hh.ru/vacancy/52558321?from=va...,HeadHunter
1,Junior Python разработчик,40000,100000,руб.,https://saratov.hh.ru/vacancy/52377613?from=va...,HeadHunter
2,Junior разработчик Python Django,40000,80000,руб.,https://saratov.hh.ru/vacancy/52203762?from=va...,HeadHunter
3,Junior Data Engineer (Python),150000,190000,руб.,https://saratov.hh.ru/vacancy/50637406?from=va...,HeadHunter
4,"Разработчик backend (Python), junior/middle",150000,NaN,руб.,https://saratov.hh.ru/vacancy/52609171?from=va...,HeadHunter
...,...,...,...,...,...,...
335,Senior Python developer,NaN,6500,USD,https://saratov.hh.ru/vacancy/51592146?from=va...,HeadHunter
336,Системный администратор,NaN,NaN,NaN,https://saratov.hh.ru/vacancy/51733070?from=va...,HeadHunter
337,Senior Python developer,NaN,6500,USD,https://saratov.hh.ru/vacancy/51592145?from=va...,HeadHunter
338,Hardware Support Engineer,1500,NaN,руб.,https://saratov.hh.ru/vacancy/51707096?from=va...,HeadHunter


In [18]:
vacancy_df.to_csv('vacancy.csv')